In [32]:
import numpy as np
import pandas as pd
import scipy.stats as sst

In [33]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.max_columns', 2000)
pd.set_option('display.max_rows', 2000)

In [34]:
def BSprice(s,k,r,q,sigma,tau,alpha):

    d1 = (np.log(s/k)+((r-q)+sigma**2/2)*tau)/(np.sqrt(tau)*sigma)
    d2 = d1-np.sqrt(tau)*sigma
    N = sst.norm(0,1).cdf
    
    return alpha*(s*np.exp(-q*tau)*N(alpha*d1)-k*np.exp(-r*tau)*N(alpha*d2))

In [35]:
def vega(s,k,r,q,sigma,tau):
    d1 = (np.log(s/k)+((r-q)+sigma**2/2)*tau)/(np.sqrt(tau)*sigma)
    d2 = d1-np.sqrt(tau)*sigma
    
    return s*np.exp(-q*tau)*sst.norm(0,1).pdf(d1)*np.sqrt(tau)

In [61]:
def imvol(s,k,r,q,tau,alpha,price):
    sigma0 = 0.1
    error = 1
    cnt = 0
    while ((error >= 1e-8) & (cnt<1000)):
        sigma1 = sigma0 - (BSprice(s,k,r,q,sigma0,tau,alpha) - price) / vega(s,k,r,q,sigma0,tau)
        error = abs(BSprice(s,k,r,q,sigma1,tau,alpha) - price)
        sigma0 = sigma1
        cnt += 1
    
    return sigma0

In [37]:
dailyopt = pd.read_csv("D://PycharmProjects/daishin/daily_opt_data.csv")
dailyopt.columns = ['code', 'S', 'K', 'r', 'tau', 'price', 'vol', 'ImVol']
dailyopt.set_index('code', inplace=True)
dailyopt['alpha'] = list(map(lambda x: 1 if x[0]=='2' else -1, dailyopt.index))

In [38]:
dailyopt.head()

,S,K,r,tau,price,vol,ImVol,alpha
code,,,,,,,,
201R3140,415.820007,140.0,0.74,17,274.549988,21.709999,64.000000,1
201R3142,415.820007,142.5,0.74,17,274.450012,21.709999,62.700001,1
201R3145,415.820007,145.0,0.74,17,271.950012,21.709999,61.400002,1
201R3147,415.820007,147.5,0.74,17,269.450012,21.709999,60.099998,1
201R3150,415.820007,150.0,0.74,17,266.950012,21.709999,58.900002,1


In [216]:
s = dailyopt['S']
k = dailyopt['K']
r = dailyopt['r']/100
q = np.zeros(len(dailyopt))
sigma = dailyopt['vol']/100
tau = dailyopt['tau']/365
alpha = dailyopt['alpha']
price_ = dailyopt['price']

In [ ]:
dailyopt['bsprice'] = BSprice(s,k,r,q,sigma,tau,alpha)
dailyopt['vega'] = vega(s,k,r,q,sigma,tau)

In [ ]:
params = np.vstack((s,k,r,q,tau,alpha,price_))
dailyopt['imvol_'] = list(map(imvol, *params))

In [85]:
dailyopt2 = dailyopt[(dailyopt['imvol_']!=inf) & (dailyopt['imvol_']!=-inf)]
dailyopt2.head()

,S,K,r,tau,price,vol,ImVol,alpha,bsprice,vega,imvol_
code,,,,,,,,,,,
201R3395,415.820007,395.0,0.74,17,23.350000,21.709999,26.860001,1,22.249159,18.968365,0.268644
201R3397,415.820007,397.5,0.74,17,21.750000,21.709999,28.299999,1,20.139248,21.878022,0.283005
201R3400,415.820007,400.0,0.74,17,20.799999,21.709999,31.430000,1,18.114892,24.762614,0.314329
201R3402,415.820007,402.5,0.74,17,19.200001,21.709999,31.830000,1,16.186134,27.513491,0.318325
201R3405,415.820007,405.0,0.74,17,15.950000,21.709999,26.830000,1,14.362253,30.019456,0.268371


In [167]:
under = dailyopt['S'][0]
dailyopt3 = dailyopt2[((dailyopt2['K']>=(under-15))&(dailyopt2['alpha']==1)) | ((dailyopt2['K']<=(under+15))&(dailyopt2['alpha']==-1))]
dailyopt3.head()

,S,K,r,tau,price,vol,ImVol,alpha,bsprice,vega,imvol_
code,,,,,,,,,,,
201R3402,415.820007,402.5,0.74,17,19.200001,21.709999,31.830000,1,16.186134,27.513491,0.318325
201R3405,415.820007,405.0,0.74,17,15.950000,21.709999,26.830000,1,14.362253,30.019456,0.268371
201R3407,415.820007,407.5,0.74,17,13.700000,21.709999,24.920000,1,12.651370,32.174489,0.249290
201R3410,415.820007,410.0,0.74,17,12.100000,21.709999,24.760000,1,11.060098,33.885416,0.247611
201R3412,415.820007,412.5,0.74,17,10.450000,21.709999,24.139999,1,9.593257,35.078725,0.241485


In [168]:
type_mat = dailyopt3[['alpha','tau']].drop_duplicates()

In [169]:
type_mat_set = []
for i in range(len(type_mat)):
    type_mat_set.append(tuple(type_mat.iloc[i]))

In [170]:
df_ = dailyopt3.groupby(['alpha','tau']).get_group(type_mat_set[0])
df_

,S,K,r,tau,price,vol,ImVol,alpha,bsprice,vega,imvol_
code,,,,,,,,,,,
201R3402,415.820007,402.5,0.74,17,19.200001,21.709999,31.830000,1,16.186134,27.513491,0.318325
201R3405,415.820007,405.0,0.74,17,15.950000,21.709999,26.830000,1,14.362253,30.019456,0.268371
201R3407,415.820007,407.5,0.74,17,13.700000,21.709999,24.920000,1,12.651370,32.174489,0.249290
201R3410,415.820007,410.0,0.74,17,12.100000,21.709999,24.760000,1,11.060098,33.885416,0.247611
201R3412,415.820007,412.5,0.74,17,10.450000,21.709999,24.139999,1,9.593257,35.078725,0.241485
201R3415,415.820007,415.0,0.74,17,8.990000,21.709999,23.770000,1,8.253666,35.705822,0.237721
201R3417,415.820007,417.5,0.74,17,7.640000,21.709999,23.379999,1,7.042045,35.746212,0.233824
201R3420,415.820007,420.0,0.74,17,6.430000,21.709999,23.049999,1,5.957017,35.208329,0.230517
201R3422,415.820007,422.5,0.74,17,5.350000,21.709999,22.740000,1,4.995209,34.128015,0.227470


In [197]:
cnt = len(df_)
price = df_['price']
code = df_.index
maxp = max(price)
minp = min(price)

In [221]:
maxPB = []
minPB = []
for i in range(cnt):
    x = np.linspace(minp/2, maxp*1.5, 50000)
    l1 = 0
    for j in range(i, cnt-1):
        if (price[i] >= min(x)) & (price[i] <= max(x)):
            if j != i:
                l1 += pow(-1, (j-i))*(price[j]-price[j+1])
            x = [t for t in x if t - price[i+1] + l1 >= 0]

            if (price[i] < min(x)) | (price[i] > max(x)):
                print('Arbitrage at stage 1')
                print('Use %s with %d upper options\n'%(code[i],j))

    
    if (i != 0):
        l2 = 0
        for j in range(i):
            if (price[i] >= min(x)) & (price[i] <= max(x)):
                if j != 0:
                    l2 += (price[i-(j+1)]-price[i-j])
                x = [t for t in x if l2 + pow(-1, j)*(price[i-1] - t) >= 0]

                if (price[i] < min(x)) | (price[i] > max(x)):
                    print('Arbitrage at stage 2')
                    print('Use %s with %d lower options\n'%(code[i],j+2))


            l3 = 0
            for k in range(i+1, cnt):
                if (price[i] >= min(x)) & (price[i] <= max(x)):
                    if k != i+1:
                        l3 += pow(-1, (k-i))*(price[k-1]-price[k])
                    x = [t for t in x if l2 + pow(-1, j)*(price[i-1] - 2*t + price[i+1]) + l3 >= 0]

                    if (price[i] < min(x)) | (price[i] > max(x)):
                        print('Arbitrage at stage 3')
                        print('Use %s with %d options, %d lower options\n'%(code[i],k-1,j+1))
                
                
    
    maxPB.append(np.round(max(x),4))
    minPB.append(np.round(min(x),4))


Arbitrage at stage 1
Use 201R3407 with 3 upper options

Arbitrage at stage 3
Use 201R3410 with 3 options, 1 lower options

Arbitrage at stage 2
Use 201R3412 with 3 lower options



In [222]:
s = df_['S']
k = df_['K']
r = df_['r']/100
q = np.zeros(len(df_))
tau = df_['tau']/365
alpha = df_['alpha']

In [223]:
df_['maxPB'] = maxPB
params2 = np.vstack((s,k,r,q,tau,alpha,maxPB))
df_['maxIV'] = list(map(imvol, *params2))
df_['minPB'] = minPB
params3 = np.vstack((s,k,r,q,tau,alpha,minPB))
df_['minIV'] = list(map(imvol, *params3))
df_ = df_[['S','K','r','tau','alpha','imvol_','price','minPB','minIV','maxPB','maxIV']]
df_

<ipython-input-61-ac98b6a9c896>:6: RuntimeWarning: divide by zero encountered in double_scalars
  sigma1 = sigma0 - (BSprice(s,k,r,q,sigma0,tau,alpha) - price) / vega(s,k,r,q,sigma0,tau)
<ipython-input-34-722472646726>:3: RuntimeWarning: invalid value encountered in double_scalars
  d1 = (np.log(s/k)+((r-q)+sigma**2/2)*tau)/(np.sqrt(tau)*sigma)


,S,K,r,tau,alpha,imvol_,price,minPB,minIV,maxPB,maxIV
code,,,,,,,,,,,
201R3402,415.820007,402.5,0.74,17,1,0.318325,19.200001,18.2501,0.287687,28.8000,0.608092
201R3405,415.820007,405.0,0.74,17,1,0.268371,15.950000,15.3002,0.247723,16.4249,0.283246
201R3407,415.820007,407.5,0.74,17,1,0.249290,13.700000,13.7502,0.250814,28.8000,0.687403
201R3410,415.820007,410.0,0.74,17,1,0.247611,12.100000,11.9102,0.242066,12.0746,0.246869
201R3412,415.820007,412.5,0.74,17,1,0.241485,10.450000,10.5000,0.242906,10.5445,0.244170
201R3415,415.820007,415.0,0.74,17,1,0.237721,8.990000,8.8501,0.233803,9.0448,0.239255
201R3417,415.820007,417.5,0.74,17,1,0.233824,7.640000,7.5301,0.230751,7.7100,0.235782
201R3420,415.820007,420.0,0.74,17,1,0.230517,6.430000,6.2901,0.226552,6.4945,0.232345
201R3422,415.820007,422.5,0.74,17,1,0.227470,5.350000,5.2505,0.224567,5.4349,0.229944
